In [9]:
import pandas as pd
import numpy as np

infile = '92.txt'
data = open(infile).read().strip()
lines = [[x.split()[0], int(x.split()[1])] for x in data.split('\n')]

n = sum([x[1] for x in lines])

diff_dict = {
    (1, 1): ['D', 'R'],
    (1, -1): ['D', 'L'],
    (-1, 1): ['U', 'R'],
    (-1, -1): ['U', 'L'],
    (0, 1): ['R'],
    (1, 0): ['D'],
    (0, -1): ['L'],
    (-1, 0): ['U'],
    (0, 0): ['O']
}

hgrid = np.zeros([2 * n + 1, 2 * n + 1])
grid9 = np.zeros([2 * n + 1, 2 * n + 1])

hi, hj = n, n
i1, j1 = n, n
i2, j2 = n, n
i3, j3 = n, n
i4, j4 = n, n
i5, j5 = n, n
i6, j6 = n, n
i7, j7 = n, n
i8, j8 = n, n
i9, j9 = n, n

hgrid[hi, hj], grid9[i9, j9] = 1, 1

def tail_loc(h1, h2, t1, t2, s):
    if s == 'R':
        if (((h1 == t1) & (h2 == t2)) | ((h1 == t1) & (h2 == t2 - 1)) |
            (((h1 == t1 + 1) | (h1 == t1 - 1)) & (h2 == t2)) |
                (((h1 == t1 + 1) | (h1 == t1 - 1)) & (h2 == t2 - 1))):
            h2 += 1
        elif ((h1 == t1) & (h2 == t2 + 1)):
            h2 += 1
            t2 += 1
        else:
            t1, t2 = h1, h2
            h2 += 1
    elif s == 'L':
        if (((h1 == t1) & (h2 == t2)) | ((h1 == t1) & (h2 == t2 + 1)) |
            (((h1 == t1 + 1) | (h1 == t1 - 1)) & (h2 == t2)) |
                (((h1 == t1 + 1) | (h1 == t1 - 1)) & (h2 == t2 + 1))):
            h2 -= 1
        elif ((h1 == t1) & (h2 == t2 - 1)):
            h2 -= 1
            t2 -= 1
        else:
            t1, t2 = h1, h2
            h2 -= 1
    elif s == 'U':
        if (((h1 == t1) & (h2 == t2)) | ((h2 == t2) & (h1 == t1 + 1)) |
            (((h2 == t2 + 1) | (h2 == t2 - 1)) & (h1 == t1)) |
                (((h2 == t2 + 1) | (h2 == t2 - 1)) & (h1 == t1 + 1))):
            h1 -= 1
        elif ((h2 == t2) & (h1 == t1 - 1)):
            h1 -= 1
            t1 -= 1
        else:
            t1, t2 = h1, h2
            h1 -= 1
    else:
        if (((h1 == t1) & (h2 == t2)) | ((h2 == t2) & (h1 == t1 - 1)) |
            (((h2 == t2 + 1) | (h2 == t2 - 1)) & (h1 == t1)) |
                (((h2 == t2 + 1) | (h2 == t2 - 1)) & (h1 == t1 - 1))):
            h1 += 1
        elif ((h2 == t2) & (h1 == t1 + 1)):
            h1 += 1
            t1 += 1
        else:
            t1, t2 = h1, h2
            h1 += 1
    #print(s, h1, h2, t1, t2)
    return h1, h2, t1, t2

def single_move(i, j, s):
    if s=='R':
        j += 1
    elif s=='L':
        j -= 1
    elif s=='U':
        i -= 1
    else:
        i += 1
    return i, j

def save_array(x, y):
    x = x[~np.all(x == 0, axis=1)]
    x = x[:, np.any(x, axis=0)]
    pd.DataFrame(x).to_csv(f'./{y}.csv')


def chk_shift(i_o, j_o, i_n, j_n):
    row_diff, col_diff = i_n - i_o, j_n - j_o
    print(row_diff, col_diff)
    steps = abs(row_diff) + abs(col_diff)
    shift_type = diff_dict[(row_diff, col_diff)]
    return shift_type, steps

def next_knot(i_o, j_o, i_n, j_n, i_next, j_next, s_):
    print(i_o, j_o, i_n, j_n, i_next, j_next, s_)
    st, steps = chk_shift(i_o, j_o, i_n, j_n)
    
    if steps == 1:
        i_o, j_o, i_next, j_next = tail_loc(i_o, j_o, i_next, j_next, st[0])
    elif steps == 2:
        s__ = s_ if s_ in st else st[0]
        i_o, j_o, i_next, j_next = tail_loc(i_o, j_o, i_next, j_next, s__)
        i_o, j_o, i_next, j_next = tail_loc(i_o, j_o, i_next, j_next,
                                            [x for x in st if x != s__][0])
    else:
        pass
    return i_next, j_next

for line in lines:
    s_ = line[0]
    print(line)
    i = 1
    while i <= line[1]:
        # Recording past values:
        i1_, j1_ = i1, j1
        i2_, j2_ = i2, j2
        i3_, j3_ = i3, j3
        i4_, j4_ = i4, j4
        i5_, j5_ = i5, j5
        i6_, j6_ = i6, j6
        i7_, j7_ = i7, j7
        i8_, j8_ = i8, j8

        hi, hj, i1, j1 = tail_loc(hi, hj, i1, j1, s_)
        i2, j2 = next_knot(i1_, j1_, i1, j1, i2, j2, s_)
        i3, j3 = next_knot(i2_, j2_, i2, j2, i3, j3, s_)
        i4, j4 = next_knot(i3_, j3_, i3, j3, i4, j4, s_)
        i5, j5 = next_knot(i4_, j4_, i4, j4, i5, j5, s_)
        i6, j6 = next_knot(i5_, j5_, i5, j5, i6, j6, s_)
        i7, j7 = next_knot(i6_, j6_, i6, j6, i7, j7, s_)
        i8, j8 = next_knot(i7_, j7_, i7, j7, i8, j8, s_)
        i9, j9 = next_knot(i8_, j8_, i8, j8, i9, j9, s_)
        
        print(hi, hj, i1, j1, i2, j2, i3, j3, i4, j4, i5, j5, i6, j6, i7, j7, i8, j8, i9, j9)
        print("\n")
        hgrid[hi, hj] = 1
        grid9[i9, i9] = 1
        i += 1

# Save into CSV files:
save_array(hgrid, 'h')
save_array(grid9, 'tail_9')
print(f'Total Positions: {np.sum(grid9)}')

['R', 5]
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 97 96 96 96 96 96 96 96 96 96 96 96 96 96 96 96 96 96 96


96 96 96 97 96 96 R
0 1
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 98 96 97 96 96 96 96 96 96 96 96 96 96 96 96 96 96 96 96


96 97 96 98 96 96 R
0 1
96 96 96 97 96 96 R
0 1
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 99 96 98 96 97 96 96 96 96 96 96 96 96 96 96 96 96 96 96


96 98 96 99 96 97 R
0 1
96 97 96 98 96 96 R
0 1
96 96 96 97 96 96 R
0 1
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 96 96 96 96 96 R
0 0
96 100 96 99 96 98 96 97 96 96 96 96 

96 110 95 110 94 110 93 109 93 108 93 107 93 106 93 105 93 104 93 103


95 110 96 110 94 110 D
1 0
94 110 95 110 93 109 D
1 0
93 109 94 110 93 108 D
1 1
93 108 94 109 93 107 D
1 1
93 107 94 108 93 106 D
1 1
93 106 94 107 93 105 D
1 1
93 105 94 106 93 104 D
1 1
93 104 94 105 93 103 D
1 1
97 110 96 110 95 110 94 110 94 109 94 108 94 107 94 106 94 105 94 104


96 110 97 110 95 110 D
1 0
95 110 96 110 94 110 D
1 0
94 110 95 110 94 109 D
1 0
94 109 94 109 94 108 D
0 0
94 108 94 108 94 107 D
0 0
94 107 94 107 94 106 D
0 0
94 106 94 106 94 105 D
0 0
94 105 94 105 94 104 D
0 0
98 110 97 110 96 110 95 110 94 109 94 108 94 107 94 106 94 105 94 104


97 110 98 110 96 110 D
1 0
96 110 97 110 95 110 D
1 0
95 110 96 110 94 109 D
1 0
94 109 95 110 94 108 D
1 1
94 108 95 109 94 107 D
1 1
94 107 95 108 94 106 D
1 1
94 106 95 107 94 105 D
1 1
94 105 95 106 94 104 D
1 1
99 110 98 110 97 110 96 110 95 110 95 109 95 108 95 107 95 106 95 105


98 110 99 110 97 110 D
1 0
97 110 98 110 96 110 D
1 0
96 110 97 

91 85 90 85 92 85 U
-1 0
92 85 91 85 93 85 U
-1 0
93 85 92 85 94 86 U
-1 0
94 86 93 85 94 87 U
-1 -1
85 85 86 85 87 85 88 85 89 85 90 85 91 85 92 85 93 85 93 86


86 85 85 85 87 85 U
-1 0
87 85 86 85 88 85 U
-1 0
88 85 87 85 89 85 U
-1 0
89 85 88 85 90 85 U
-1 0
90 85 89 85 91 85 U
-1 0
91 85 90 85 92 85 U
-1 0
92 85 91 85 93 85 U
-1 0
93 85 92 85 93 86 U
-1 0
84 85 85 85 86 85 87 85 88 85 89 85 90 85 91 85 92 85 93 86


85 85 84 85 86 85 U
-1 0
86 85 85 85 87 85 U
-1 0
87 85 86 85 88 85 U
-1 0
88 85 87 85 89 85 U
-1 0
89 85 88 85 90 85 U
-1 0
90 85 89 85 91 85 U
-1 0
91 85 90 85 92 85 U
-1 0
92 85 91 85 93 86 U
-1 0
83 85 84 85 85 85 86 85 87 85 88 85 89 85 90 85 91 85 92 85


84 85 83 85 85 85 U
-1 0
85 85 84 85 86 85 U
-1 0
86 85 85 85 87 85 U
-1 0
87 85 86 85 88 85 U
-1 0
88 85 87 85 89 85 U
-1 0
89 85 88 85 90 85 U
-1 0
90 85 89 85 91 85 U
-1 0
91 85 90 85 92 85 U
-1 0
82 85 83 85 84 85 85 85 86 85 87 85 88 85 89 85 90 85 91 85


83 85 82 85 84 85 U
-1 0
84 85 83 85 85 85 U
-1 0
8